##  Sentinel-2 burned area identification

### Service Definition

The service definition is defined with a dictionnary with:

- **id**, the processing parameter identifier
- **title**, the title for the processing parameter
- **abstract**, the description of the processing parameter

Example:

```python

service = dict([('title', 'My nice app'),
                ('abstract', 'Application to process ...'),
                ('id', 'ewf-some-app')])
```

In [17]:
service = dict([('title', 'Sentinel-2 burned area identification'),
                ('abstract', 'This is a short description'),
                ('id', 'ewf-satcen-03-03-02')])

### Parameter Definition 

Processing parameters are defined with a dictionnary with:

- **id**, the processing parameter identifier
- **value**, the default value for the processing parameter
- **title**, the title for the processing parameter
- **abstract**, the description of the processing parameter

Example:

```python

ts = dict([('id', 'ts'),
           ('value', '1000'),
           ('title', 'Training set size'),
           ('abstract', 'Size of the training set (in pixels)')])
```

### Runtime parameter definition

**Input identifier**

The input identifier is the catalogue entry identifier value.

It can be easily obtained in a shell with, for example:

```bash
opensearch-client 'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL1C_20170909T060631_N0205_R134_T42SVG_20170909T061506' identifier
$ S2A_MSIL1C_20170909T060631_N0205_R134_T42SVG_20170909T061506
```

In [12]:
input_identifier = 'S2A_MSIL1C_20170909T060631_N0205_R134_T42SVG_20170909T061506'

**Input reference**

The input identifier is the catalogue entry URL (a.k.a. self value).

In [7]:
input_reference = 'https://catalog.terradue.com/sentinel2/search?uid=S2A_MSIL1C_20170909T060631_N0205_R134_T42SVG_20170909T061506'

**Data path**

This path defines where the data is staged-in. 

In [5]:
data_path = '/workspace/data'

### Workflow

#### Import the packages

In [13]:
import os
import sys
import cioppy
import geopandas as gp

In [14]:
ciop = cioppy.Cioppy()

In [15]:
input_metadata = gp.GeoDataFrame(ciop.search(end_point=input_reference, 
                                params=[],
                                output_fields='identifier,self,wkt,startdate,enddate,enclosure', 
                                model='EOP'))

In [16]:
input_metadata

,enclosure,enddate,identifier,self,startdate,wkt
0,http://scihub.terradue.com/apihub/Products('21...,2017-09-09T06:06:31.0260000Z,S2A_MSIL1C_20170909T060631_N0205_R134_T42SVG_2...,https://catalog.terradue.com/sentinel2/search?...,2017-09-09T06:06:31.0260000Z,"POLYGON((67.8614709212009 37.9420809165272,69...."


Other code blocks

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.